# Harzardous Liquid Incidents Cleaning and Analysis
There are two csv files about harzardous liquid spills in the US. The one with geocodes includes location information other than longtitude and longitude of the incidents.

In [27]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_liquid = pd.read_csv('data/hazardous_liquid_clean.csv')
df_liquid_geo = pd.read_csv('data/hazardous_liquid_geocodioc.csv')

In [3]:
df_liquid['net_spill'] = df_liquid['UNINTENTIONAL_RELEASE_BBLS'].fillna(0) + df_liquid['INTENTIONAL_RELEASE_BBLS'].fillna(0) - df_liquid['RECOVERED_BBLS'].fillna(0)

In [4]:
df_liquid['net_spill'] = df_liquid['net_spill'].fillna(0)
df_liquid.to_csv('df_liquid.csv')

In [5]:
df_liquid.head()

,IYEAR,OPERATOR_ID,NAME,LOCAL_DATETIME,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,COMMODITY_SUBTYPE,COMMODITY_DETAILS,UNINTENTIONAL_RELEASE_BBLS,...,USA_DRINKING_YES_NO,EST_COST_GAS_RELEASED,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_ENVIRONMENTAL,EST_COST_OTHER,PART_C_QUESTION_2_IND,CAUSE,CAUSE_DETAILS,net_spill
0,2021,30829,ENTERPRISE CRUDE PIPELINE LLC,6/6/21 8:19,28.946250,-95.423650,CRUDE OIL,NaN,NaN,1.5,...,YES,0.0,2000.0,21000.0,10000.0,0.0,NO,OTHER INCIDENT CAUSE,UNKNOWN,0.0
1,2021,39596,PERMIAN EXPRESS PARTNERS LLC,6/5/21 1:00,29.989279,-94.002547,CRUDE OIL,NaN,NaN,4.0,...,NaN,240.0,1000.0,500.0,2000.0,0.0,NO,EQUIPMENT FAILURE,DEFECTIVE OR LOOSE TUBING OR FITTING,0.0
2,2021,31476,"ROSE ROCK MIDSTREAM OPERATING, LLC",6/4/21 10:39,35.927207,-96.760283,CRUDE OIL,NaN,NaN,0.5,...,NaN,0.0,35.0,3000.0,2500.0,1500.0,NaN,NaN,EQUIPMENT FAILURE,0.0
3,2021,4906,EXXONMOBIL PIPELINE CO,5/30/21 12:50,29.843158,-95.230803,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,GASOLINE (NON-ETHANOL),NaN,640.0,...,YES,1400.0,1140000.0,2608000.0,1631000.0,0.0,YES,MATERIAL FAILURE OF PIPE OR WELD,"CONSTRUCTION-, INSTALLATION-, OR FABRICATION-R...",590.0
4,2021,26061,"DELEK CRUDE LOGISTICS, LLC.",5/24/21 14:49,32.479730,-94.836680,CRUDE OIL,NaN,NaN,15.0,...,NaN,1000.0,10000.0,10000.0,0.0,0.0,NO,EQUIPMENT FAILURE,MALFUNCTION OF CONTROL/RELIEF EQUIPMENT,0.0


In [6]:
df_liquid['net_spill'].sum()

1056484.29

In [7]:
df_liquid_geo['INTENTIONAL_RELEASE_BBLS'] = df_liquid_geo['INTENTIONAL_RELEASE_BBLS'].fillna(0)

In [8]:
df_liquid_geo['UNINTENTIONAL_RELEASE_BBLS']= df_liquid_geo['UNINTENTIONAL_RELEASE_BBLS'].fillna(0)

In [9]:
df_liquid_geo['RECOVERED_BBLS']= df_liquid_geo['RECOVERED_BBLS'].fillna(0)

In [10]:
df_liquid_geo['net_spill'] = df_liquid_geo['UNINTENTIONAL_RELEASE_BBLS'] + df_liquid_geo['INTENTIONAL_RELEASE_BBLS'] - df_liquid_geo['RECOVERED_BBLS']

In [11]:
df_liquid_geo.to_csv('df_liquid_geo.csv')

###  Step 1: List pipeline operators

In [12]:
operators = df_liquid.NAME.value_counts()
operator_incident = operators.to_frame().reset_index().rename(columns={'index':'PIPE LINE OPERATOR','NAME':'INCIDENT'})

In [13]:
operator_damage = df_liquid[['NAME','EST_COST_PROP_DAMAGE']].groupby('NAME').sum('EST_COST_PROP_DAMAGE').reset_index().rename(columns={'EST_COST_PROP_DAMAGE':'PROPERTY DAMAGE COST'})

In [14]:
operator_incident_cost = operator_incident.merge(operator_damage, left_on = 'PIPE LINE OPERATOR', right_on='NAME').drop(columns = 'NAME', axis = 1)
operator_incident_cost

,PIPE LINE OPERATOR,INCIDENT,PROPERTY DAMAGE COST
0,ENTERPRISE CRUDE PIPELINE LLC,319,29632944.0
1,ENTERPRISE PRODUCTS OPERATING LLC,254,38232652.0
2,SUNOCO PIPELINE L.P.,239,12806907.0
3,"PLAINS PIPELINE, L.P.",234,16855003.0
4,"MAGELLAN PIPELINE COMPANY, LP",209,7894714.0
...,...,...,...
319,"DRY TRAILS MIDSTREAM ENERGY, LLC",1,109592.0
320,TARGA RESOURCES OPERATING LLC,1,50000.0
321,"STAKEHOLDER MIDSTREAM CRUDE OIL PIPELINE, LLC",1,17229.0
322,DOW PIPELINE CO,1,0.0


In [15]:
operator_incident_cost.to_csv('operator_incident_cost.csv')

### Step 2: get the number of incident of each liquid type

In [16]:
df_liquid.COMMODITY_RELEASED_TYPE.value_counts()

CRUDE OIL                                                                             2278
REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHICH IS A LIQUID AT AMBIENT CONDITIONS    1435
HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS A GAS AT AMBIENT CONDITIONS             709
CO2 (CARBON DIOXIDE)                                                                    63
BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL BLENDS)                                     5
Name: COMMODITY_RELEASED_TYPE, dtype: int64

In [32]:
crude_oil = df_liquid[df_liquid['COMMODITY_RELEASED_TYPE'] == 'CRUDE OIL']
crude_oil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 4488
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   IYEAR                         2278 non-null   int64  
 1   OPERATOR_ID                   2278 non-null   int64  
 2   NAME                          2278 non-null   object 
 3   LOCAL_DATETIME                2278 non-null   object 
 4   LOCATION_LATITUDE             2278 non-null   float64
 5   LOCATION_LONGITUDE            2278 non-null   float64
 6   COMMODITY_RELEASED_TYPE       2278 non-null   object 
 7   COMMODITY_SUBTYPE             0 non-null      object 
 8   COMMODITY_DETAILS             0 non-null      object 
 9   UNINTENTIONAL_RELEASE_BBLS    2278 non-null   float64
 10  INTENTIONAL_RELEASE_BBLS      428 non-null    float64
 11  RECOVERED_BBLS                2278 non-null   float64
 12  FATAL                         2 non-null      float64
 13  INJ

In [29]:
crude_oil_year = crude_oil.IYEAR.value_counts().to_frame().reset_index().rename(columns={'index':'Year','IYEAR':'Crude Oil Incident'})
crude_oil_year

,Year,Crude Oil Incident
0,2015,257
1,2014,241
2,2018,221
3,2017,208
4,2013,205
5,2016,204
6,2019,197
7,2012,189
8,2020,166
9,2010,156


In [ ]:
crude_oil['']

In [30]:
petroleum = df_liquid[df_liquid['COMMODITY_RELEASED_TYPE'] == 'REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHICH IS A LIQUID AT AMBIENT CONDITIONS']
petroleum_year = petroleum.IYEAR.value_counts().to_frame().reset_index().rename(columns={'index':'Year','IYEAR':'Petroleum Incident'})
petroleum_year

,Year,Petroleum Incident
0,2014,158
1,2012,134
2,2016,134
3,2013,133
4,2015,133
5,2010,125
6,2011,123
7,2017,122
8,2019,117
9,2018,110


In [31]:
hvl = df_liquid[df_liquid['COMMODITY_RELEASED_TYPE'] == 'HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS A GAS AT AMBIENT CONDITIONS']
hvl_year = hvl.IYEAR.value_counts().to_frame().reset_index().rename(columns={'index':'Year','IYEAR':'HVL Incident'})
hvl_year

,Year,HVL Incident
0,2017,76
1,2016,72
2,2011,71
3,2018,67
4,2019,66
5,2020,64
6,2010,63
7,2015,63
8,2013,57
9,2014,50


In [44]:
df_liquid[df_liquid['FATAL']>0].FATAL

263     5.0
1643    1.0
1759    2.0
1771    1.0
2346    1.0
3327    1.0
3634    1.0
3732    2.0
4313    1.0
Name: FATAL, dtype: float64

In [45]:
df_liquid[df_liquid['INJURE']>0].INJURE

175     7.0
263     4.0
284     1.0
925     2.0
1643    1.0
1759    4.0
1771    1.0
1852    4.0
3105    2.0
3152    3.0
3352    1.0
3634    1.0
3732    3.0
3799    1.0
4153    1.0
4313    1.0
4396    1.0
Name: INJURE, dtype: float64

### Step 3: break things down by states: TX, OK, LA, CA, KS...
how many incidents in each state

In [16]:
# Find out which states have more incident.
incident_state = df_liquid_geo.State.value_counts().to_frame().reset_index()
incident_state.head()

,index,State
0,TX,1727
1,OK,379
2,LA,285
3,CA,239
4,KS,202


In [51]:
spill_state = df_liquid_geo.groupby('State').sum('net_spill').sort_values(by = 'net_spill', ascending = False).reset_index()
spill_state = spill_state[['State','net_spill']]
spill_state.head()


,State,net_spill
0,TX,427608.95
1,LA,162730.30
2,MS,126981.94
3,WV,46118.95
4,OK,43688.75


In [54]:
# Save the results to csv file for future visualzation and analysis.
liquid_top_states = pd.DataFrame(
    {'state': df_liquid_geo.State.value_counts().index, 
     'number of incidents':df_liquid_geo.State.value_counts().values,
     'net spill':spill_state.net_spill})
liquid_top_states
liquid_top_states.to_csv('liquid_top_states.csv')

In [19]:
def by_state(state):
    return df_liquid_geo[df_liquid_geo['State'] == state]

In [20]:
df_texas = by_state('TX')
df_oklahoma = by_state('OK')
df_louisiana = by_state('LA')
df_california = by_state('CA')
df_kansas = by_state('KS')
df_illinois = by_state('IL')
df_newjersy = by_state('NY')
df_wyoming = by_state('WY')
df_newmexico = by_state('NM')
df_northdakota = by_state('ND')

In [21]:
def state_year(state, year):
    return df_liquid_geo[(df_liquid_geo['State'] == state)&(df_liquid_geo['IYEAR'] == year)]

In [194]:
texas_2020 = state_year('TX', (2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020))
texas_2019 = state_year('TX', (2010,2011,2012,2013,2014,2015,2016,2017,2018,2019))
texas_2018 = state_year('TX', (2010,2011,2012,2013,2014,2015,2016,2017,2018))
texas_2017 = state_year('TX', (2010,2011,2012,2013,2014,2015,2016,2017))
texas_2016 = state_year('TX', (2010,2011,2012,2013,2014,2015,2016))
texas_2015 = state_year('TX', (2010,2011,2012,2013,2014,2015))
texas_2014 = state_year('TX', (2010,2011,2012,2013,2014))
texas_2013 = state_year('TX', (2010,2011,2012,2013))
texas_2012 = state_year('TX', (2010,2011,2012))
texas_2011 = state_year('TX', (2010,2011))
texas_2010 = state_year('TX', 2010)

In [195]:
texas_2010.to_csv('texas_2010.csv')
texas_2011.to_csv('texas_2011.csv')
texas_2012.to_csv('texas_2012.csv')
texas_2013.to_csv('texas_2013.csv')
texas_2014.to_csv('texas_2014.csv')
texas_2015.to_csv('texas_2015.csv')
texas_2016.to_csv('texas_2016.csv')
texas_2017.to_csv('texas_2017.csv')
texas_2018.to_csv('texas_2018.csv')
texas_2019.to_csv('texas_2019.csv')
texas_2020.to_csv('texas_2020.csv')

######Create two new dataframes for future visualization:
1. net_spill by year and state
2. number of incidents by year and state

In [25]:
spill_year_states = df_liquid_geo[['IYEAR','State','net_spill']].groupby(['IYEAR','State']).sum('net_spill').reset_index()
spill_year_states = spill_year_states.pivot_table(index = 'State', columns = 'IYEAR', values = 'net_spill').fillna(0).reset_index()
spill_year_states.head()

IYEAR,State,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AK,1119.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
1,AL,0.00,1.00,0.00,0.0,581.00,9.52,9061.77,4.76,0.00,50.00,0.10,0.0
2,AR,0.00,0.43,1.00,1391.2,1.10,13500.20,2.40,0.00,3.87,0.33,58.32,0.0
3,CA,162.88,127.57,22.68,35.0,399.32,2162.95,164.89,8.10,0.19,9.17,931.01,1210.5
4,CO,4.99,0.12,0.48,2.0,2106.11,1301.10,3837.68,945.70,0.00,434.30,197.40,0.0


In [26]:
spill_year_states.to_csv('spill_year_states.csv')

In [27]:
df_liquid_geo['incident_count'] = 1
incident_year_state = df_liquid_geo[['IYEAR','State','incident_count']].groupby(['IYEAR','State']).sum('incident_count').reset_index()
incident_year_state = incident_year_state.pivot_table(index = 'State', columns = 'IYEAR', values = 'incident_count').fillna(0).reset_index()
incident_year_state.head()

IYEAR,State,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AK,1.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0
1,AL,1.0,2.0,2.0,3.0,2.0,2.0,8.0,3.0,2.0,1.0,3.0,1.0
2,AR,1.0,1.0,1.0,4.0,3.0,1.0,2.0,2.0,4.0,2.0,6.0,1.0
3,CA,15.0,24.0,22.0,17.0,28.0,25.0,23.0,19.0,17.0,21.0,18.0,10.0
4,CO,3.0,1.0,2.0,2.0,8.0,4.0,4.0,7.0,4.0,7.0,5.0,0.0


In [28]:
incident_year_state.to_csv('incident_year_state.csv')

### Step 4: break the data down again, this time, by year from 2010- 2020

###### First, let's figure out the numbers of incidents, fatal cases, injure cases, and property damage by year

In [58]:
incident_count = df_liquid.IYEAR.value_counts().to_frame().reset_index().rename(columns={'IYEAR':'year','incident':'Incident'})
incident_count

,year,Incident
0,2015,460
1,2014,455
2,2016,420
3,2017,415
4,2018,405
5,2013,400
6,2019,384
7,2012,366
8,2010,350
9,2011,344


In [59]:
fatal_count = df_liquid.groupby('IYEAR').sum('FATAL').reset_index()
other_count = fatal_count[['IYEAR','FATAL','INJURE','EST_COST_PROP_DAMAGE']]
year_count = other_count.merge(incident_count,left_on='IYEAR', right_on="year")
year_count.rename(columns={'IYEAR':'Year','FATAL':'Fatal','INJURE':'Injure','EST_COST_PROP_DAMAGE':'Estimated Property Damage $'}).drop(columns='year',axis =1)

,Year,Fatal,Injure,Estimated Property Damage $,Incident
0,2010,1.0,3.0,54535020.0,350
1,2011,0.0,1.0,45400844.0,344
2,2012,3.0,4.0,40401399.0,366
3,2013,1.0,6.0,43616841.0,400
4,2014,0.0,0.0,47362115.0,455
5,2015,1.0,0.0,46834985.0,460
6,2016,3.0,9.0,58459078.0,420
7,2017,1.0,1.0,46140710.0,415
8,2018,0.0,2.0,39871926.0,405
9,2019,0.0,0.0,30899405.0,384


In [60]:
year_count.to_csv('year_count.csv')

In [34]:
cause_year = pd.DataFrame(df_liquid.groupby('IYEAR').CAUSE.value_counts()).rename(columns={'CAUSE': 'Incident'}).reset_index().pivot_table(index = 'IYEAR', columns = 'CAUSE', values = 'Incident').reset_index()
cause_year.to_csv('cause_year.csv')
#= commodity_year.rename(columns={'IYEAR':'Year','BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL BLENDS)':'Biofuel/Alternative Fuel', 'CO2 (CARBON DIOXIDE)':'CO2','CRUDE OIL':'Crude Oil', 'HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS A GAS AT AMBIENT CONDITIONS':'HVL','REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHICH IS A LIQUID AT AMBIENT CONDITIONS':'Refined Petroleum'}).fillna(0)

In [37]:
df_liquid.sort_values(by='EST_COST_PROP_DAMAGE', ascending=False).head(10)

,IYEAR,OPERATOR_ID,NAME,LOCAL_DATETIME,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,COMMODITY_SUBTYPE,COMMODITY_DETAILS,UNINTENTIONAL_RELEASE_BBLS,...,USA_DRINKING_YES_NO,EST_COST_GAS_RELEASED,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_ENVIRONMENTAL,EST_COST_OTHER,PART_C_QUESTION_2_IND,CAUSE,CAUSE_DETAILS,net_spill
4356,2010,26149,ALYESKA PIPELINE SERVICE CO,5/25/10 11:00,63.930740,-145.766610,CRUDE OIL,NaN,NaN,2580.00,...,NaN,238000.0,27000000.0,1700000.0,0.0,0.0,NO,INCORRECT OPERATION,"TANK, VESSEL, OR SUMP/SEPARATOR ALLOWED OR CAU...",1119.00
2658,2014,31554,CHEVRON PETROCHEMICAL PIPELINE LLC,10/10/14 12:30,30.050300,-93.721048,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,OTHER HVL,ETHYLENE,6.00,...,NaN,0.0,24323483.0,0.0,0.0,0.0,YES,MATERIAL FAILURE OF PIPE OR WELD,ORIGINAL MANUFACTURING-RELATED (NOT GIRTH WELD...,6.00
4132,2011,26149,ALYESKA PIPELINE SERVICE CO,1/8/11 8:16,70.256660,-148.620300,CRUDE OIL,NaN,NaN,308.00,...,NaN,0.0,14130000.0,1570000.0,0.0,0.0,NO,CORROSION FAILURE,INTERNAL CORROSION,0.00
3785,2012,31174,"SHELL PIPELINE CO., L.P.",1/14/12 7:10,42.951700,-87.896200,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,"DIESEL, FUEL OIL, KEROSENE, JET FUEL",NaN,215.00,...,YES,11000.0,11614000.0,4125000.0,200000.0,0.0,YES,CORROSION FAILURE,EXTERNAL CORROSION,87.00
2388,2015,300,"PLAINS PIPELINE, L.P.",5/19/15 10:57,34.462434,-120.086714,CRUDE OIL,NaN,NaN,2934.00,...,NaN,144000.0,9868173.0,90701042.0,22421933.0,19796736.0,YES,CORROSION FAILURE,EXTERNAL CORROSION,1834.00
1990,2016,32147,MARATHON PIPE LINE LLC,4/16/16 3:00,38.273730,-87.851660,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,"DIESEL, FUEL OIL, KEROSENE, JET FUEL",NaN,854.00,...,NaN,44800.0,7701361.0,1750789.0,0.0,0.0,YES,NATURAL FORCE DAMAGE,HEAVY RAINS/FLOODS,854.00
1766,2016,30829,ENTERPRISE CRUDE PIPELINE LLC,10/23/16 21:51,35.940287,-96.749203,CRUDE OIL,NaN,NaN,7603.00,...,NaN,22500.0,7446138.0,350000.0,0.0,0.0,YES,EXCAVATION DAMAGE,PREVIOUS DAMAGE DUE TO EXCAVATION ACTIVITY,276.00
3925,2011,31618,ENTERPRISE PRODUCTS OPERATING LLC,8/13/11 1:57,42.004610,-96.236840,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,OTHER,NATURAL GASOLINE,675.00,...,NO,80595.0,7368470.0,135000.0,85000.0,0.0,YES,NATURAL FORCE DAMAGE,HEAVY RAINS/FLOODS,675.00
1324,2017,32334,TC OIL PIPELINE OPERATIONS INC,11/16/17 5:33,45.709444,-97.876944,CRUDE OIL,NaN,NaN,6592.00,...,NaN,583560.0,5682477.0,6042594.0,31677529.0,0.0,YES,MATERIAL FAILURE OF PIPE OR WELD,"CONSTRUCTION-, INSTALLATION-, OR FABRICATION-R...",0.00
875,2018,2552,COLONIAL PIPELINE CO,12/31/18 7:30,35.285832,-80.932844,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,"DIESEL, FUEL OIL, KEROSENE, JET FUEL",NaN,407.14,...,NaN,29011.0,5490339.0,3002960.0,2346524.0,0.0,NO,CORROSION FAILURE,INTERNAL CORROSION,77.33


###### Second, let's figure out the ratios of released commodities by year. For example, how many percentage of crude oil in 2015...

In [186]:
commodity_year = pd.DataFrame(df_liquid.groupby('IYEAR').COMMODITY_RELEASED_TYPE.value_counts()).rename(columns={'COMMODITY_RELEASED_TYPE': 'Incident'}).reset_index().pivot_table(index = 'IYEAR', columns = 'COMMODITY_RELEASED_TYPE', values = 'Incident').reset_index()
commodity_year = commodity_year.rename(columns={'IYEAR':'Year','BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL BLENDS)':'Biofuel/Alternative Fuel', 'CO2 (CARBON DIOXIDE)':'CO2','CRUDE OIL':'Crude Oil', 'HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS A GAS AT AMBIENT CONDITIONS':'HVL','REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHICH IS A LIQUID AT AMBIENT CONDITIONS':'Refined Petroleum'}).fillna(0)

In [113]:
commodity_year['total']= commodity_year['Biofuel/Alternative Fuel']+commodity_year['CO2']+commodity_year['Crude Oil']+commodity_year['HVL'] + commodity_year['Refined Petroleum']
commodity_year

COMMODITY_RELEASED_TYPE,Year,Biofuel/Alternative Fuel,CO2,Crude Oil,HVL,Refined Petroleum,total
0,2010,0.0,6.0,156.0,63.0,125.0,350.0
1,2011,0.0,4.0,146.0,71.0,123.0,344.0
2,2012,0.0,2.0,189.0,41.0,134.0,366.0
3,2013,0.0,5.0,205.0,57.0,133.0,400.0
4,2014,1.0,5.0,241.0,50.0,158.0,455.0
5,2015,0.0,7.0,257.0,63.0,133.0,460.0
6,2016,1.0,9.0,204.0,72.0,134.0,420.0
7,2017,0.0,9.0,208.0,76.0,122.0,415.0
8,2018,2.0,5.0,221.0,67.0,110.0,405.0
9,2019,0.0,4.0,197.0,66.0,117.0,384.0


In [114]:
commodity_year.to_csv('commodity_year.csv')

In [115]:
commodity_year['Biofuel/Alternative Fuel Percentage'] = commodity_year['Biofuel/Alternative Fuel']/commodity_year['total'] *100
commodity_year['CO2 Percentage'] =commodity_year['CO2']/commodity_year['total'] *100
commodity_year['Crude Oil Percentage'] =commodity_year['Crude Oil']/commodity_year['total'] *100
commodity_year['HVL Percentage'] =commodity_year['HVL']/commodity_year['total'] *100
commodity_year['Refined Petroleum Percentage'] =commodity_year['Refined Petroleum']/commodity_year['total'] *100
commodity_percentage_year = commodity_year.drop(columns=['Biofuel/Alternative Fuel','total','Biofuel/Alternative Fuel','CO2','Crude Oil','HVL','Refined Petroleum'], axis = 1)
commodity_percentage_year

COMMODITY_RELEASED_TYPE,Year,Biofuel/Alternative Fuel Percentage,CO2 Percentage,Crude Oil Percentage,HVL Percentage,Refined Petroleum Percentage
0,2010,0.000000,1.714286,44.571429,18.000000,35.714286
1,2011,0.000000,1.162791,42.441860,20.639535,35.755814
2,2012,0.000000,0.546448,51.639344,11.202186,36.612022
3,2013,0.000000,1.250000,51.250000,14.250000,33.250000
4,2014,0.219780,1.098901,52.967033,10.989011,34.725275
5,2015,0.000000,1.521739,55.869565,13.695652,28.913043
6,2016,0.238095,2.142857,48.571429,17.142857,31.904762
7,2017,0.000000,2.168675,50.120482,18.313253,29.397590
8,2018,0.493827,1.234568,54.567901,16.543210,27.160494
9,2019,0.000000,1.041667,51.302083,17.187500,30.468750


In [110]:
commodity_percentage_year.to_csv('commdity_percentage_year.csv')

In [150]:
# 1. 2021
liquid_2021 = df_liquid[df_liquid['IYEAR'] == 2021]
liquid_2021.to_csv('liquid_2021.csv')

In [151]:
# 2. 2020
liquid_2020 = df_liquid[df_liquid['IYEAR'] == 2020]
liquid_2020.to_csv('liquid_2020.csv')

In [152]:
# 3. 2019
liquid_2019 = df_liquid[df_liquid['IYEAR'] == 2019]
liquid_2019.to_csv('liquid_2019.csv')

In [153]:
# 4. 2018
liquid_2018 = df_liquid[df_liquid['IYEAR'] == 2018]
liquid_2018.to_csv('liquid_2018.csv')

In [154]:
# 5. 2017
liquid_2017 = df_liquid[df_liquid['IYEAR'] == 2017]
liquid_2017.to_csv('liquid_2017.csv')

In [155]:
# 6. 2016
liquid_2016 = df_liquid[df_liquid['IYEAR'] == 2016]
liquid_2016.to_csv('liquid_2016.csv')

In [156]:
# 7. 2015
liquid_2015 = df_liquid[df_liquid['IYEAR'] == 2015]
liquid_2015.to_csv('liquid_2015.csv')

In [157]:
# 8. 2014
liquid_2014 = df_liquid[df_liquid['IYEAR'] == 2014]
liquid_2014.to_csv('liquid_2014.csv')

In [158]:
# 9. 2013
liquid_2013 = df_liquid[df_liquid['IYEAR'] == 2013]
liquid_2013.to_csv('liquid_2013.csv')

In [159]:
# 10. 2012
liquid_2012 = df_liquid[df_liquid['IYEAR'] == 2012]
liquid_2012.to_csv('liquid_2012.csv')

In [160]:
# 11. 2011
liquid_2011 = df_liquid[df_liquid['IYEAR'] == 2011]
liquid_2011.to_csv('liquid_2011.csv')

In [161]:
# 12. 2010
liquid_2010 = df_liquid[df_liquid['IYEAR'] == 2010]
liquid_2010.to_csv('liquid_2010.csv')

# Serious Liquid Incidents Cleaning and Analysis

Now we are going to take a look at the serious incidents of hazardous liquid. 

First, we need to filter the hazardous liquid from the full dataset and save it to a new csv file.

Then, we can dig into it.

In [29]:
df_serious_incidents= pd.read_csv('data/pipline_serious_incidents.csv')

In [53]:
df_serious_incidents.head()

,Calendar Year,Portal Year,Incident Cause Type,Incident Cause SubType,Serious Incident Flag,Significant Incident Flag,Unnamed: 6,# of Fatalities,# of Injuries,Total Property Damage,Total Property Damage-Cur Yr Dollars,Commodity Spilled Amount bbls,Net Spill bbls,System Sub Type,Serious Incident Flag.1,Serious Incident Flag.2,Serious Incident Flag.3,State Name
0,2001,7/8/21,ALL OTHER CAUSES,MISCELLANEOUS,No,Yes,1,0,0,750000.0,1.078689e+06,0.0,0.0,GAS DISTRIBUTION,No,No,No,CONNECTICUT
1,2001,7/8/21,ALL OTHER CAUSES,MISCELLANEOUS,No,Yes,1,0,0,250000.0,3.595629e+05,0.0,0.0,GAS DISTRIBUTION,No,No,No,MINNESOTA
2,2001,7/8/21,ALL OTHER CAUSES,MISCELLANEOUS,No,Yes,1,0,0,170000.0,2.445028e+05,0.0,0.0,GAS DISTRIBUTION,No,No,No,OHIO
3,2001,7/8/21,ALL OTHER CAUSES,MISCELLANEOUS,No,Yes,1,0,0,200000.0,2.876503e+05,0.0,0.0,GAS DISTRIBUTION,No,No,No,VIRGINIA
4,2001,7/8/21,ALL OTHER CAUSES,MISCELLANEOUS,No,Yes,1,0,0,100000.0,1.438252e+05,0.0,0.0,GAS TRANSMISSION,No,No,No,ALASKA


In [35]:
df_serious_liquid = df_serious_incidents[df_serious_incidents['System Sub Type'] == 'HAZARDOUS LIQUID']

In [54]:
df_serious_liquid.shape

(1864, 18)

In [38]:
df_serious_liquid['Incident Cause Type'].value_counts()

MATERIAL/WELD/EQUIP FAILURE    760
CORROSION                      320
INCORRECT OPERATION            227
EXCAVATION DAMAGE              217
NATURAL FORCE DAMAGE           140
ALL OTHER CAUSES               109
OTHER OUTSIDE FORCE DAMAGE      91
Name: Incident Cause Type, dtype: int64

In [55]:
df_serious_liquid['State Name'].value_counts().head(10)

TEXAS         328
LOUISIANA     156
OKLAHOMA      149
CALIFORNIA    119
KANSAS        116
ILLINOIS      112
NEW MEXICO     58
WYOMING        56
OHIO           55
MINNESOTA      49
Name: State Name, dtype: int64

In [51]:
serious_liquid_state = pd.DataFrame({'state':df['State Name'].value_counts().index, 'incidents':df['State Name'].value_counts().values })
serious_liquid_state.to_csv('serious_liquid_state.csv')